In [ ]:
%matplotlib inline
#git clone https://github.com/AllonKleinLab/scrublet.git
#cd scrublet
#sudo pip3 install -r requirements.txt
#sudo pip3 install --upgrade .
#sudo pip3 install pySankeyBeta
##remember to do sudo and pip3 otherwise it doesn't work
#import scrublet as scr
import scipy.io
import random
#import scvelo as scv
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
import seaborn as sns
import numpy as np
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfTransformer
import sys
sys.path.append("/Users/konstas/ScanpyPlus")  # Replace with the actual path
import Scanpyplus
from importlib import reload
from scanpy.tools._utils import get_init_pos_from_paga as init
import pandas as pd
#import pandasPlus


import scanpy as sc
import anndata
#import bbknn
import os
from scipy import sparse

In [ ]:
adata = sc.read_h5ad("HCA_kidney_sc_manual.h5ad")
adata

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.neighbors(adata, use_rep='X_scVI')
sc.tl.leiden(adata)

In [ ]:
sc.pl.embedding(adata[adata.obs['pct_counts_ribo']>30], basis='X_scVI_umap', color=['leiden', 'pct_counts_ribo'], legend_loc='on data')

## Proximal tubule annotation

In [ ]:
PT_genelist = ['NEAT1', 'LRP2', 'LINC00621', 'ACSM2A', 'BNC2', 'ACSM2B', 'SLC13A3', 'LINC01320', 'NLGN1', 'CUBN', 'LRP2', 'SLC13A1','COL4A1', 'VCAM1', 'AKAP12', 'BICC1', 'LINC01320', 'CREB5', 'TJP1', 'VCAM1', 'NRP1', 'DCDC2']
PTS1_genelist = ['SLC5A12', 'SLC13A3', 'SLC22A6', 'PRODH2', 'SLC5A2', 'SLC22A8', 'SLC5A12', 'SLC13A3', 'SLC22A6', 'PRODH2', 'SLC5A2', 'SLC22A8', 'AL031599.1', 'AC105094.2', 'RNF212B', 'PCDH15', 'CDH20', 'ACSM2B', 'ACSM2A', 'SLC36A2', 'SLC5A12', 'SLC16A9', 'CYP3A5', 'LINC01060', 'CUBN', 'NPL', 'AP003400.1', 'NOX4', 'AFM', 'PAH', 'SLC4A4', 'PPP2R2B']
PTS2_genelist = ['SLC5A12' , 'SLC13A3', 'SLC22A6', 'SLC34A1', 'SLC22A7', 'ACSM2A', 'CUBN', 'ACSM2B', 'LRP2', 'AC096577.1', 'SLC13A3', 'SORCS1', 'SLC5A12', 'SLC4A4', 'TINAG', 'SLC16A9', 'SLC34A1', 'RAB11FIP3', 'HNF4A', 'AFM', 'SLC22A6', 'AC004053.1', 'SLC17A1', 'AC087762.1', 'ZNF804B']
PTS3_genelist = ['SLC22A7', 'MOGAT1', 'SLC5A11', 'SLC22A24', 'SLC7A13', 'SLC5A8', 'ABCC3', 'SATB2', 'SLC22A7', 'MOGAT1', 'SLC5A11', 'SLC22A24', 'SLC7A13', 'SLC5A8', 'ABCC3', 'SATB2', 'C5orf67', 'AL078590.3', 'SLC7A13', 'SLC30A8', 'ANKS1B', 'AC084128.1', 'GPM6A', 'SLC22A24', 'AL049629.1', 'LINC01789', 'ABCC3', 'AC128707.1', 'AL078590.2']

aPT_score = ['CDH6', 'TNFSF10', 'COL4A1', 'MACF1', 'TJP1', 'GPC4', 'VCAM1', 'AKAP12',  
'COL4A2', 'ANXA4', 'DCDC2', 'NRP1', 'ITGAV', 'NTN4', 'BICC1', 'MACC1',  
'PAPLN', 'PPFIBP1', 'LINC01320', 'RETREG1', 'OXR1', 'ABCB1', 'CRIM1',  
'CREB5', 'FLNB', 'JADE1'
]

sc.tl.score_genes(adata, PT_genelist, n_bins=25, score_name='score_PT')
sc.tl.score_genes(adata, PTS1_genelist, n_bins=25, score_name='score_PTS1')
sc.tl.score_genes(adata, PTS2_genelist, n_bins=25, score_name='score_PTS2')
sc.tl.score_genes(adata, PTS3_genelist, n_bins=25, score_name='score_PTS3')
sc.tl.score_genes(adata, aPT_score, n_bins=30, score_name='score_aPT')

In [ ]:
PTclusters_with_high_score = adata.obs.loc[adata.obs['score_PT'] > 2, 'leiden'].unique()
PTclusters_with_high_score

In [ ]:
PTS1clusters_with_high_score = adata.obs.loc[adata.obs['score_PTS1'] > 2, 'leiden'].unique()
PTS1clusters_with_high_score
PTS1_dict = {cluster: 'PTS1/2' for cluster in PTS1clusters_with_high_score}
PTS1_dict 

In [ ]:
PTS2clusters_with_high_score = adata.obs.loc[adata.obs['score_PTS2'] > 2, 'leiden'].unique()
PTS2clusters_with_high_score
PTS2_dict = {cluster: 'PTS1/2' for cluster in PTS2clusters_with_high_score}
PTS2_dict 

In [ ]:
PTS3clusters_with_high_score = adata.obs.loc[adata.obs['score_PTS1'] > 2, 'leiden'].unique()
PTS3clusters_with_high_score
PTS3_dict = {cluster: 'PTS3' for cluster in PTS3clusters_with_high_score}
PTS3_dict

## Podocytes & Parietal epithelial cells

In [ ]:
Podo_genelist = ['PTPRQ', 'WT1', 'NTNG1', 'NPHS1', 'NPHS2', 'CLIC5', 'PODXL', 
 'FMN2', 'ADAMTS19', 'TARID', 'ATP10A', 'AC092813.2', 'CR1', 'ZNF804A', 
 'ALS2CL', 'NFASC', 'PCOLCE2', 'ARHGEF26', 'LINC02149', 'FGF1', 'WDR49', 
 'AC244021.1', 'PTPRO', 'ST6GALNAC3', 'PLA2R1', 'FYN', 'CTGF', 'SRGAP2C', 
 'PLCE1', 'KIRREL1', 'CDC42EP3', 'CDC14A', 'SRGAP2B']

Podo_cellxgene = ['PODXL', 'CLIC5', 'PTPRO', 'NPHS2', 'MME', 'NPHS1', 'ST6GALNAC3', 'WT1', 
 'ARMH4', 'DACH1', 'ST3GAL6', 'PLA2R1', 'MPP5', 'TGFBR3', 'PTH1R', 'THSD7A', 
 'ENPEP', 'NTNG1', 'NEBL', 'SRGAP1', 'PLCE1', 'MAGI2', 'VEGFA', 'IQGAP2', 
 'RIPOR1']

PEC = ["ALDH1A2", "FRMD4A", "PDE1A", "SYNE1", "RBFOX1", "LINC01435", "MAGI2", "OSBPL10", 
"DCDC2", "BICC1", "NRG3", "CFH", "ZFPM2", "BICD1", "BNC2", "RERG", "CCDC198", 
"LRP2", "CRIM1", "KCNT2", "CTTNBP2", "TENM3", "ITGB8", "THRB", "PARD3B"
]

sc.tl.score_genes(adata, Podo_cellxgene, n_bins=25, score_name='score_PODCxG')
sc.tl.score_genes(adata, Podo_genelist, n_bins=25, score_name='score_POD')
sc.tl.score_genes(adata, PEC, n_bins=30, score_name='score_PEC')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_POD >1], basis='X_scVI_umap', color=['score_POD', 'score_PODCxG', 'leiden'], ncols=2, legend_loc = 'on data')

In [ ]:
PODclusters_with_high_score = adata.obs.loc[adata.obs['score_POD'] > 1, 'leiden'].unique()
PODclusters_with_high_score

In [ ]:
POD_dict = {cluster: 'POD' for cluster in PODclusters_with_high_score}
POD_dict #43 

## Pericytes

In [ ]:
PC_genelist = ['NR2F2-AS1', 'MEIS1', 'PALLD', 'H19', 'EBF1', 'COL1A1', 
 'AUTS2', 'DLC1', 'FBXL7', 'RBMS3', 'PDE3A', 'MEIS2', 'GPC6', 'KIAA1217', 
 'DPP6', 'ROR1', 'IL1RAPL1', 'CDH11', 'FLRT2', 'PRKG1', 'IGF2', 'NR2F2', 
 'COL1A2', 'SYT1', 'PCDH9', 'CLDN1', 'VCAM1', 'CFH', 'RBFOX1', 'ALDH1A2']
renal_PC = ['PRKG1', 'CACNA1C', 'EBF1', 'NR2F2-AS1', 'RBMS3', 'PDE3A', 'SOX5', 'CARMN', 'CALD1', 'MYH11', 'ZFHX3', 'PDGFRB', 'RCAN2', 'TNS1', 'HIP1', 'RGS5_ENSG00000143248', 'DLC1', 'PLCL1', 'NOTCH3', 'TMTC1', 'LHFPL6', 'INPP4B', 'CACNB2', 'NTRK3', 'PIP5K1B'
]
sc.tl.score_genes(adata, PC_genelist, n_bins=25, score_name='score_PC')
sc.tl.score_genes(adata, renal_PC, n_bins=25, score_name='renal_PC')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_PEC > 1], basis='X_scVI_umap', color=['score_PEC', 'renal_PEC', 'leiden', 'KCNJ8'], legend_loc= 'on data', ncols=2)

In [ ]:
PCclusters_with_high_score = adata.obs.loc[adata.obs['score_PEC'] > 15, 'leiden'].unique()
PCclusters_with_high_score

In [ ]:
PC_dict = {cluster: 'PEC' for cluster in PECclusters_with_high_score}
PC_dict #9, 41

## Macula densa

In [ ]:
MD_genelist = ['NOS1', 'ROBO2', 'CALCR', 'PPFIA2', 'PAPPA2', 'COX2', 'NOX2', 'NOX4']
sc.tl.score_genes(adata, MD_genelist, n_bins=25, score_name='score_MD')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_MD > 2], basis='X_scVI_umap', color=['score_MD', 'leiden' ], ncols=2, legend_loc= 'on data')

In [ ]:
MDclusters_with_high_score = adata.obs.loc[adata.obs['score_MD'] > 2, 'leiden'].unique()
MDclusters_with_high_score

In [ ]:
MD_dict = {cluster: 'MD' for cluster in MDclusters_with_high_score}
MD_dict #0 some cells

## Distal convoluted tubule

In [ ]:
DCT_genelist = ['SLC12A3', 'AC078980.1', 'TRPM6', 'KLHL3', 'LINC01055', 'TMEM52B', 'DEPDC1B', 
 'FMN1', 'CACNB4', 'SALL3', 'PID1', 'TBC1D9', 'CNNM2', 'ITPKB', 'TCF24', 'WNK1', 
 'TRIM50', 'EGF', 'TRPM7', 'LHX1', 'SLC4A1', 'SLC26A7', 'HS6ST3', 'NXPH2', 'LEF1', 
 'ADGRF5', 'FGF13', 'ADAMTS17', 'ZNF385D', 'HS6ST2', 'TRPV5', 'SLC8A1', 'SCN2A', 
 'HSD11B2', 'CALB1', 'CA12', 'KCNJ16', 'AC013463.2', 'ESRRG', 'KNG1',
 'PTGER3', 'CCSER1', 'ATP1A1', 'ATP1B1', 'MECOM', 'ARHGAP6', 'CACNA2D3', 'GPC5', 'RP11-368L12.1', 'LINC01762', 'ABCA5']
sc.tl.score_genes(adata, DCT_genelist, n_bins=30, score_name='score_DCT')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_DCT > 1.5], basis='X_scVI_umap', color=['score_DCT', 'leiden'], ncols=2, legend_loc= 'on data')

In [ ]:
DCTclusters_with_high_score = adata.obs.loc[adata.obs['score_DCT'] > 20, 'leiden'].unique()
DCTclusters_with_high_score

In [ ]:
DCT_dict = {cluster: 'DCT' for cluster in DCTclusters_with_high_score}
DCT_dict #8, 27

## Papillary

In [ ]:
PapE_genelist = [
    "KRT19", "TACSTD2", "ELF3", "CRYBG1", "PPARG", "MYOF", "MIR205HG", "AHR", "ATP8B1", "ZBTB7C", "SEMA5A", "SPINK1",
    "INPP4B", "CD2AP", "SMAD3", "PTGR1", "MCC", "LINC00511", "PSD3", "KLF5", "AQP3", "SERINC5", "DGKH", "YAP1", "KRT7"
]
sc.tl.score_genes(adata, PapE_genelist, n_bins=25, score_name='score_PapE')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_PapE > 1.7], basis='X_scVI_umap', color=['score_PapE', 'leiden', 'Project_ID'], ncols=2, legend_loc= 'on data')

In [ ]:
PapEclusters_with_high_score = adata.obs.loc[adata.obs['score_PapE'] > 25, 'leiden'].unique()
PapEclusters_with_high_score

In [ ]:
PapE_dict = {cluster: 'PapE' for cluster in PapEclusters_with_high_score}
PapE_dict #34

## Connecting tubule

In [ ]:
CNT_ICA_genelist = ['SLC4A1', 'SLC26A7', 'SLC8A1', 'SCN2A', 'CALB1', 'ATP6V0D2', 'SLC26A7', 'PTGER3', 'ADGRF5', 'ARHGAP18', 'CLNK', 'ATP6V0A4', 
 'TMEM213', 'CA12', 'TMEM101', 'ATP6V1C2', 'RCAN2', 'ITGA6', 'GRB14', 'PACRG', 
 'LINC01187', 'PDE1C', 'TFCP2L1', 'TMEM117', 'ABCA5', 'ATP6V1A', 'ABCC4', 
 'SLIT2', 'GALNT17', 'TBC1D14']
CNT_genelist = ['AC013463.2', 'CADPS2', 'MECOM', 'ESRRG', 'HSD11B2', 'CCSER1', 'PIK3C2G', 'PDE4D', 
 'SCN2A', 'COBLL1', 'KCNIP4', 'TEX41', 'WNK1', 'NR3C2', 'ARL15', 'PTH2R', 'PKHD1', 
 'PRKG1', 'MPPED2', 'CA12', 'DCDC2', 'TOX3', 'TRPM3', 'FHOD3', 'CALB1']
CNT_PC = ['SCNN1G', 'RALYL', 'KCNIP1', 'SCNN1B', 'KITLG', 'SLC8A1', 'CALB1', 'TOX', 'HSD11B2', 'SGPP1', 'SCN2A', 'PCDH7','DEFB1', 'CDH16', 'HSD11B2', 'TMEM52B', 'GSTM3', 'RHCG', 'CLCNKB', 'KCNJ1', 
 'CALB1', 'SCPEP1', 'CA12', 'EPCAM', 'CA2', 'ATP6V1B1', 'ATP1B1', 'TACSTD2', 
 'MAL', 'TMEM213', 'AIF1L', 'RALBP1', 'WNK1', 'WFDC2', 'ATP1A1', 'PTGR1', 'CLDN7']

sc.tl.score_genes(adata, CNT_genelist, n_bins=25, score_name='score_CNT')
sc.tl.score_genes(adata, CNT_PC, n_bins=25, score_name='score_CNT-PC')
sc.tl.score_genes(adata, CNT_ICA_genelist, n_bins=25, score_name='score_CNT-IC-A')

In [ ]:
sc.pl.embedding(adata[adata.obs['score_CNT-PC'] >1], basis='X_scVI_umap', color=['score_CNT', 'score_CNT-PC', 'score_CNT-IC-A', 'leiden', 'Project_ID'], ncols=2, legend_loc= 'on data')

In [ ]:
CNTclusters_with_high_score = adata.obs.loc[adata.obs['score_CNT'] > 20, 'leiden'].unique()
CNTclusters_with_high_score

In [ ]:
CNT_dict = {cluster: 'CNT' for cluster in CNTclusters_with_high_score}
CNT_dict #27

In [ ]:
CNTPCclusters_with_high_score = adata.obs.loc[adata.obs['score_CNT-PC'] > 30, 'leiden'].unique()
CNTPCclusters_with_high_score

In [ ]:
CNTPC_dict = {cluster: 'CNT-PC' for cluster in CNTPCclusters_with_high_score}
CNTPC_dict #8

In [ ]:
CNTICAclusters_with_high_score = adata.obs.loc[adata.obs['score_CNT-IC-A'] > 20, 'leiden'].unique()
CNTICAclusters_with_high_score

In [ ]:
CNTICA_dict = {cluster: 'CNT-IC-A' for cluster in CNTICAclusters_with_high_score}
CNTICA_dict #part of 17

## Collecting duct

In [ ]:
CD_genelist = ['AQP2', 'STC1', 'BMPR1B', 'HSD11B2', 'GATA3', 'PIK3C2G', 'PWRN1', 'MECOM', 'COBLL1', 'SLC38A1', 'AQP3', 'ADAMTS16', 'MPPED2', 'CADPS2', 'CRYBG1', 'L1CAM', 'TOX', 'TACSTD2', 'LHX1', 'TEX41', 'GATA3', 'HSD11B2', 'AQP2', 'MAL', 'FXYD4', 'AC013463.2', 'TMEM213', 'SCIN', 'CDH16', 
 'PIK3C2G', 'ATP6V0A4', 'MPPED2', 'FYB2', 'ATP1B1', 'ATP6V0D2', 'BMPR1B', 
 'C11orf80', 'MUC1', 'SLC26A7', 'PTGER3', 'TFCP2L1', 'SCNN1G', 'PWRN1', 
 'CA12', 'FAM107B', 'MECOM']
               
CD_PC_genelist = ['AC013463.2', 'KCNK13', 'FXYD4', 'MUC1', 'WFDC2', 'AQP2', 'CDH16', 
 'AIF1L', 'FAM107B', 'SCIN', 'PDE10A', 'SCNN1G', 'ST6GAL1', 'ATP1B1', 
 'PROM1', 'HSD11B2', 'EPCAM', 'RALBP1', 'SLC14A2', 'FAM81A', 'PIK3C2G', 
 'SYK', 'HS3ST5', 'SCNN1B', 'BMPR1B', 'SOX5', 'MAL', 'PCDH7', 'COBLL1', 
 'PWRN1', 'FYB2', 'SLC25A29', 'KRT19', 'EYA4', 'PHACTR1', 'C11orf80', 'PAPPA']

CD_ICA_genelist = ['ATP6V0D2', 'TMEM213', 'ATP6V0A4', 'SLC26A7', 'CA12', 'PTGER3', 'CLNK', 'RCAN2', 
 'ADGRF5', 'PDE1C', 'TFCP2L1', 'PACRG', 'MAGI1', 'ATP6V1C2', 'SCIN', 'MUC20', 
 'EPS8', 'SNTB1', 'MAL', 'SLC4A1', 'NEDD4L', 'TMEM117', 'ATP6V1A', 'AC013463.2', 'ESRRG']

ICB_genelist = [
    'CA12', 'SLC4A9', 'ATP6V0D2', 'ATP6V0A4', 'LINC01187', 'PLCG2', 'IL18', 
    'TFCP2L1', 'RNF152', 'SLC26A4', 'CLNK', 'WDR72', 'RCAN2', 'EPS8', 'TMEM213', 
    'PACRG', 'MAGI3', 'MAGI1', 'IQGAP2', 'NEDD4L', 'PDE1C', 'TMEM117', 'PDZD2', 
    'CA8', 'KRT7', 'SAMHD1', 'SLC35F3', 'NTRK1', 'TLDC2', 'AL136369.2', 'AC090502.1', 
    'SFTA1P', 'AC008438.1', 'CELF2', 'DGKI', 'PPM1E', 'AL355472.3', 'AC007563.2', 
    'ATP6V1C2', 'INSRR']

sc.tl.score_genes(adata, CD_genelist, n_bins=25, score_name='score_CD')
sc.tl.score_genes(adata, CD_PC_genelist, n_bins=25, score_name='score_CD-PC')
sc.tl.score_genes(adata, CD_ICA_genelist, n_bins=25, score_name='score_CD-ICA')
sc.tl.score_genes(adata, ICB_genelist, n_bins=25, score_name='score_ICB')

In [ ]:
sc.pl.embedding(adata[adata.obs['score_CD'] > 1], basis='X_scVI_umap', color=['score_CD', 'score_CD-PC', 'score_CD-ICA', 'score_ICB', 'leiden'], ncols=2, legend_loc= 'on data')

In [ ]:
sc.pl.embedding(adata[adata.obs.leiden == '38'], basis='X_scVI_umap', color=['score_CD', 'score_CD-PC', 'score_CD-ICA', 'score_ICB', 'score_CNT-IC-A', 'score_CNT-PC', 'leiden']) 

In [ ]:
CDclusters_with_high_score = adata.obs.loc[adata.obs['score_CD'] > 15, 'leiden'].unique()
CDclusters_with_high_score

In [ ]:
CD_dict = {cluster: 'CD' for cluster in CDclusters_with_high_score}
CD_dict 

In [ ]:
CDPCclusters_with_high_score = adata.obs.loc[adata.obs['score_CD-PC'] > 20, 'leiden'].unique()
CDPCclusters_with_high_score

In [ ]:
CDPC_dict = {cluster: 'CD-PC' for cluster in CDPCclusters_with_high_score}
CDPC_dict #11

In [ ]:
CDICclusters_with_high_score = adata.obs.loc[adata.obs['score_CD-ICA'] > 20, 'leiden'].unique()
CDICclusters_with_high_score 

In [ ]:
CDIC_dict = {cluster: 'CD-ICA' for cluster in CDICclusters_with_high_score}
CDIC_dict # part 38

In [ ]:
CDICBclusters_with_high_score = adata.obs.loc[adata.obs['score_ICB'] > 20, 'leiden'].unique()
CDICBclusters_with_high_score 

In [ ]:
CDICB_dict = {cluster: 'CD-ICB' for cluster in CDICBclusters_with_high_score}
CDICB_dict #part of 38

## Loop of henle

In [ ]:
LH_genelist = ['SLC12A1', 'ERBB4', 'GPC5', 'UMOD', 'PLCB1', 'PDE1A', 'PCDH9', 'KCNIP4', 'LINC01762', 'MECOM', 'RP1', 'ACPP', 'ESRRG', 'CASR', 'ENOX1', 'EGF', 'ESRRB', 'CCSER1', 'PKP4', 'HS6ST2', 'CRYAB', 'TACSTD2', 'SLC44A5', 'KLRG2', 'COL26A1', 'BOC']

ATL_genelist = ['MAL', 'TACSTD2', 'ATP1A1', 'CLDN10', 'FXYD2', 'SLPI', 'MUC1', 'WFDC2', 'KRT7', 
 'SPP1', 'CXCL14', 'MMP7', 'SLC12A1', 'SLC5A3', 'CLDN4', 'CLDN3', 'CD24',
 'SOD3', 'S100A2', 'FOXP2', 'CYS1', 'CHL1', 'EPCAM', 'SH3GL3', 'ERMP1', 'CLDN1', 'AKR1B1', 'CLDN4', 'BCL6', 'SH3GL3', 'BCAS1', 'CLCNKA', 'CLDN10', 'PROX1']


DTL_genelist = [
    "SLC12A3", "TRPM6", "AC019197.1", "SLC8A1", "TEX41", "CNNM2", "KLHL3", "SCN2A", "EFNA5", "KCNIP4", "CACNB4", "ERBB4", 
    "ESRRG", "EGF", "KITLG", "FMN1", "PIK3C2G", "CPEB4", "TRPM7", "MECOM", "VCAM1", "SLC39A8", "AQP1", "LRRC4C", "LRP2", 
    "UNC5D", "SATB2", "CLDN1", "AKR1B1", "CLDN4", "BCL6", "SH3GL3", "SLC14A2", "SMOC2", "JAG1", "ADGRL3", "ID1", 
    "CFH", "LINC01435", "SLC4A11", "LINC00924", "KIRREL3", "KLRG2", "LINC01197", "MSC-AS1", "ALDH1A2", "KCNT2", 
    "AC098617.1", "GRM5", "MGAT4C", "FBXL7", "HIST1H2AC", "TSHZ2", "ZFPM2", "AC003984.1", "SLIT3"
]
 


TAL_genelist = ['CASR', 'SLC12A1', 'UMOD', 'NELL1', 'ESRRB', 'EGF', 'CLDN14', 'PROX1', 'MFSD4A', 
 'KCTD16', 'RAP1GAP', 'ANK2', 'CYFIP2', 'PPM1E', 'GP2', 'ENOX1', 'TMEM207', 
 'TMEM52B', 'CLDN16', 'WNK1', 'ATP1A1', 'KCNJ16', 'ESRRG', 'ATP1B1', 'KNG1', 
 'CACNA2D3', 'PLCB1', 'PTGER3', 'CLCN5', 'PDE1A', 'CA12', 'CCSER1', 'CGNL1', 
 'KCNJ1', 'MECOM', 'KCNIP4', 'PTH2R', 'GPC5', 'SIM2', 'BICDL1']

score_aTAL = ['RNF19A', 'VMP1', 'GLS', 'ELF3', 'ITGB1', 'RAPGEF3', 'ITGB6', 'EZR',  
'ITGA3', 'SYNE1', 'CLDN4', 'TMPRSS4', 'NEAT1', 'PAX8', 'DST', 'SAT1',  
'AHNAK', 'MIDN', 'KRT7', 'RNF213', 'TFPI', 'SKIL', 'PLEKHA1'
]

sc.tl.score_genes(adata, LH_genelist, n_bins=25, score_name='score_LH')
sc.tl.score_genes(adata, ATL_genelist, n_bins=25, score_name='score_ATL')
sc.tl.score_genes(adata, DTL_genelist, n_bins=25, score_name='score_DTL')
sc.tl.score_genes(adata, TAL_genelist, n_bins=25, score_name='score_TAL')
sc.tl.score_genes(adata, score_aTAL, n_bins=30, score_name='score_aTAL')

In [ ]:
LHclusters_with_high_score = adata.obs.loc[adata.obs['score_LH'] > 20, 'leiden'].unique()
LHclusters_with_high_score

In [ ]:
ATLclusters_with_high_score = adata.obs.loc[adata.obs['score_ATL'] > 20, 'leiden'].unique()
ATLclusters_with_high_score

In [ ]:
ATL_dict = {cluster: 'ATL' for cluster in ATLclusters_with_high_score}
ATL_dict #8, 21

In [ ]:
DTLclusters_with_high_score = adata.obs.loc[adata.obs['score_DTL'] > 20, 'leiden'].unique()
DTLclusters_with_high_score 

In [ ]:
DTL_dict = {cluster: 'DTL' for cluster in DTLclusters_with_high_score}
DTL_dict #36

In [ ]:
TALclusters_with_high_score = adata.obs.loc[adata.obs['score_TAL'] > 20, 'leiden'].unique()
TALclusters_with_high_score

In [ ]:
TAL_dict = {cluster: 'TAL' for cluster in TALclusters_with_high_score}
TAL_dict #18, 10

In [ ]:
adata.obs['annotation_v1'] = adata.obs['leiden'].map(epithelial_dict)

In [ ]:
# mitochondrial genes, "MT-" for human, "Mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var_names.str.contains("^HB[^(P)]")
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt", "ribo", "hb"], inplace=True, log1p=True
)

In [ ]:
adata

In [ ]:
high_ribo_score = adata.obs.loc[adata.obs['pct_counts_ribo'] > 90, 'leiden'].unique()
high_ribo_score

In [ ]:
high_mt_score = adata.obs.loc[adata.obs['pct_counts_mt'] > 90, 'leiden'].unique()
high_mt_score

In [ ]:
sc.pl.embedding(adata, basis='X_scVI_umap', color=['leiden', 'pct_counts_mt', 'pct_counts_ribo'], ncols=1, legend_loc= 'on data')

In [ ]:
adata.write("Downloads/Kidney_Combined_hvg15k_sc_annotation.h5ad")

In [ ]:
adata

## Endothelial cells

In [ ]:
EC_genelist = ['CD34', 'PECAM1', 'PTPRB', 'MEIS2', 'FLT1', 'EMCN', 'MEIS2', 'PTPRB', 'FLT1', 'NOTCH4', 'PLAT', 'EGFL7', 'AC010737.1', 'ERG', 'SLCO2A1', 'RAPGEF4', 'EMCN', 'BTNL9', 'PECAM1', 'PREX2', 'RUNX1T1', 'CDH13', 'KLF2', 'GFOD1', 'LDB2', 'ITGA8']
EC_AEA = ["BTNL9", "ADAMTS6", "PALMD", "AQP1", "TM4SF1", "VEGFC", "CCDC3", "CDH5", "SERPINE2", "FBLN5", "CXCL12", "SOX17"]
EC_DVR	= ["BTNL9", "ADAMTS6", "PALMD", "AQP1", "TM4SF1", "MCTP1", "SLC14A1", "ENPP2", "LYPD6B"]
EC_PTC	= ["CEACAM1", "DNASE1L3", "PLVAP", "PITPNC1", "GRB10", "SLCO2A1", "RAPGEF4"]
EC_AVR	= ["CEACAM1", "DNASE1L3", "PLVAP", "GPM6A", "EDIL3", "TLL1", "ZNF385D", "NR2F2"]
EC_LYM	= ["MMRN1", "CD36", "TBX1", "PKHD1L1", "PROX1"]
EC_cap = ["EPAS1", "PLPP1", "PTPRB", "EMCN", "SLC9A3R2", "TIMP3", "FLT1", "ADGRF5", "PECAM1", "EGFL7", "CAVIN2", "RNASE1", "RAMP2", "PLAT", "SEC14L1", "TM4SF1", "ENG", "PLPP3", "ADGRL4", "RAMP3", "NOSTRIN", "TACC1", "SPTBN1", "IL6ST", "SLCO2A1"]
glomerular_CAP=["EMCN", "PLAT", "RAMP2", "IGFBP5", "CRHBP", "GNG11", "TGFBR2", "PLPP3", "SLC9A3R2", "PLPP1", "RNASE1", "EPAS1", "TM4SF1", "SOST", "PTPRB", "EHD3", "ADGRF5", "FLT1", "VAMP5", "MGP", "EGFL7", "CAVIN2", "HES1", "KDR", "LDB2"]



sc.tl.score_genes(adata, EC_genelist, n_bins=30, score_name='score_EC')
sc.tl.score_genes(adata, EC_AEA, n_bins=30, score_name='score_EC_AEA')
sc.tl.score_genes(adata, EC_DVR, n_bins=30, score_name='score_EC_DVR')
sc.tl.score_genes(adata, EC_PTC, n_bins=30, score_name='score_EC_PTC')
sc.tl.score_genes(adata, EC_AVR, n_bins=30, score_name='score_EC_AVR')
sc.tl.score_genes(adata, EC_LYM, n_bins=30, score_name='score_EC_LYM')
sc.tl.score_genes(adata, EC_cap, n_bins=30, score_name='score_EC_CAP')
sc.tl.score_genes(adata, glomerular_CAP, n_bins=30, score_name='score_EC-glomerular')

In [ ]:
sc.pl.embedding(adata[adata.obs.score_EC > 1.5], basis='X_scVI_umap', color=['score_EC', 'score_EC_AEA', 'score_EC_DVR', 'score_EC_PTC', 'score_EC_AVR', 'score_EC_LYM', 'SULF1','leiden'], ncols=2, legend_loc= 'on data')

In [ ]:
ECclusters_with_high_score = adata.obs.loc[adata.obs['score_EC'] > 20, 'leiden'].unique()
ECclusters_with_high_score

In [ ]:
ECAEAclusters_with_high_score = adata.obs.loc[adata.obs['score_EC_AEA'] > 30, 'leiden'].unique()
ECAEAclusters_with_high_score #23

In [ ]:
ECDVRclusters_with_high_score = adata.obs.loc[adata.obs['score_EC_DVR'] > 20, 'leiden'].unique()
ECDVRclusters_with_high_score #23 part of

In [ ]:
ECAVRclusters_with_high_score = adata.obs.loc[adata.obs['score_EC_AVR'] > 20, 'leiden'].unique()
ECAVRclusters_with_high_score 

In [ ]:
ECPTCclusters_with_high_score = adata.obs.loc[adata.obs['score_EC_PTC'] > 30, 'leiden'].unique()
ECPTCclusters_with_high_score 

In [ ]:
ECLYMclusters_with_high_score = adata.obs.loc[adata.obs['score_EC_LYM'] > 20, 'leiden'].unique()
ECLYMclusters_with_high_score 

## Mesangial, VSMC, Fibroblasts

In [ ]:
MC_genelist = ['PIP5K1B', 'ROBO1', 'PIEZO2', 'DAAM2', 'PHTF2', 'GATA3', 'POSTN', 'NOTCH3', 'PDGFRB', 'ITGA8', 'CARMN', 'EBF1', 'PDGFRB', 'NTRK3', 'ITGA8', 'SYNPO2', 'ADIRF', 'LMOD1', 'MYOCD', 'MICAL2', 'DAAM2', 'ADCY3', 'MEIS2', 'COL25A1', 'DGKG', 'CPED1', 'KALRN', 'TMTC1', 'RBMS3', 'NOTCH3']
MC_CxG =['IGF2', 'H19_ENSG00000130600', 'COL1A1', 'NR2F2-AS1', 'MEIS1', 'GPC3', 'PALLD', 'AUTS2', 'DPP6', 'COL1A2', 'KIAA1217', 'FLRT2', 'MEIS2', 'COL3A1', 'FBXL7', 'IL1RAPL1', 'AFF3', 'NR2F2', 'CDH11', 'SYT1', 'ROBO2', 'TNC', 'LINC00486_ENSG00000230876', 'LDB2', 'CCBE1'
]
sc.tl.score_genes(adata, MC_genelist, n_bins=30, score_name='score_MC')
sc.tl.score_genes(adata, MC_CxG, n_bins=30, score_name='MC_CxG')

In [ ]:
MCclusters_with_high_score = adata.obs.loc[adata.obs['score_MC'] > 10, 'leiden'].unique()
MCclusters_with_high_score

In [ ]:
VSMC_genelist = ["TPM2", "CALD1", "BGN", "MYL9", "ACTA2", "TAGLN", "THY1", "RGS5", "COL6A2", "PLAC9", "FRZB", "MFGE8", "CPE", "TPM4", "SERPINH1", "C11orf96", "COL1A2", "LGALS1", "MCAM", "MGP", "ISYNA1", "GJA4", "ID3", "TPM1", "MAGED2"]

sc.tl.score_genes(adata, VSMC_genelist, n_bins=30, score_name='score_VSMC')

In [ ]:
fibro_genelist = ['RBMS3', 'NR2F2-AS1', 'MEIS1', 'C7_ENSG00000112936', 'PRKG1', 'CALD1', 'ATRNL1', 'SOX5', 'ITGA1', 'TSHZ2', 'THSD4', 'CACNA1C', 'PALLD', 'NEGR1', 'CDH11', 'SVEP1', 'LDB2', 'TRERF1', 'GNA14', 'EBF1', 'SETBP1', 'PDGF'
]

sc.tl.score_genes(adata, fibro_genelist, n_bins=30, score_name='score_fibro')

In [ ]:
Fibroclusters_with_high_score = adata.obs.loc[adata.obs['score_fibro'] > 10, 'leiden'].unique()
Fibroclusters_with_high_score

In [ ]:
VSMCclusters_with_high_score = adata.obs.loc[adata.obs['score_fibro'] > 10, 'leiden'].unique()
VSMCclusters_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_VSMC > 10], basis='X_scVI_umap', color=['score_VSMC', 'leiden'], ncols=2, legend_loc= 'on data')

In [ ]:
sc.pl.embedding(adata, basis='X_scVI_umap', color=['annotation_genemarker', 'leiden', 'pct_counts_ribo'], ncols=1, legend_loc='on data')

## Macrophage

In [ ]:
Macro_genelist = ["C1QA", "C1QB", "C1QC", "MS4A6A", "NPC2", "SELENOP", "CD14", "HLA-DPA1", "MS4A7", "HLA-DRA", "TYROBP", "HLA-DRB1", "FTL", "PSAP", "HLA-DPB1", "CST3", "FCGRT", "HLA-DQB1", "CTSB", "HLA-DQA1", "HLA-DMA", "SAT1", "CYBA", "CD74", "FCER1G"]

sc.tl.score_genes(adata, Macro_genelist, n_bins=30, score_name='score_Macro')

In [ ]:
Macroclusters_with_high_score = adata.obs.loc[adata.obs['score_Macro'] > 10, 'leiden'].unique()
Macroclusters_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_Macro > 0], basis='X_scVI_umap', color=['score_Macro', 'leiden', 'pct_counts_mt'], ncols=2, legend_loc= 'on data')

In [ ]:
Mono_genelist = ["FCN1", "CTSS", "SERPINA1", "TYROBP", "CST3", "S100A9", "LYZ", "SAT1", "CFD", "PSAP", "TYMP", "C1orf162", "LST1", "IFI30", "CD68", "S100A8", "MS4A7", "S100A11", "FTL", "CFP", "LILRB2", "FCGR3A", "COTL1", "PILRA", "SLC11A1"]


sc.tl.score_genes(adata, Mono_genelist, n_bins=30, score_name='score_Mono')

Monoclusters_with_high_score = adata.obs.loc[adata.obs['score_Mono'] > 10, 'leiden'].unique()
Monoclusters_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_Mono > 1], basis='X_scVI_umap', color=['score_Mono', 'leiden', 'pct_counts_mt', 'FCGR3A'], ncols=2, legend_loc= 'on data')

In [ ]:
DC_genelist = ["HLA-DPB1", "HLA-DPA1", "HLA-DRA", "HLA-DQB1", "HLA-DRB1", "CST3", "HLA-DQA1", "HLA-DMA", "C1orf162", "MS4A6A", "CD74", "LYZ", "FGL2", "CPVL", "CLEC10A", "PPT1", "NPC2", "GRN", "RGS10", "HLA-DRB5", "FCER1A", "HLA-DMB", "TYROBP", "IGSF6", "C1QA"]

sc.tl.score_genes(adata, DC_genelist, n_bins=30, score_name='score_DC')

DC_with_high_score = adata.obs.loc[adata.obs['score_DC'] > 20, 'leiden'].unique()
DC_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_DC > 20], basis='X_scVI_umap', color=['score_DC', 'leiden', 'pct_counts_mt', "CD40", "CD83", "HLA-DRA", "ITGAX", "LYZ"], ncols=2, legend_loc= 'on data')

In [ ]:
Lympho_gene_list = ["CD3E", "GZMA", "CST7", "CD3D", "IL2RG", "CD2", "NKG7", "PTPRCAP", "CTSW", "RAC2", "IL32", "GZMK", "CD7", "LCK", "EVL", "CD69", "CCL5", "CD8A", "DUSP2", "PTPRC", "PLAAT4", "CD3G", "CD27", "CD247", "PRF1"]

Tnaive_gene_list = ["GZMK", "CD3D", "CD8A", "CST7", "NKG7", "CD3E", "GZMA", "PTPRCAP", "CD8B", "CD27", "HLA-A", "CD2", "HLA-C", "B2M", "HLA-B", "IL2RG", "CD3G", "LAG3", "CTSW", "APOBEC3G", "IL32", "PSMB9", "SIRPG", "LCK", "PRF1"]
CD4_Ef_Mem_gene_list = ["IL7R", "CD3D", "CD3E", "CD2", "IL32", "FLT3LG", "ZFP36L2", "CD3G", "LCK", "CD52", "IL2RG", "CD40LG", "OXNAD1", "CORO1B", "BTG1", "CD6", "ITK", "IFITM1", "SPOCK2", "GZMA", "CRIP1", "CD5", "TTC39C", "TRAC", "CYTIP"]
CD8_Ef_Mem_gene_list = ["CCL5", "NKG7", "CST7", "GZMA", "CD3D", "CD3E", "CD8A", "GZMK", "CD2", "CCL4", "CTSW", "CD8B", "PRF1", "IL32", "CD3G", "CD27", "IL2RG", "HLA-B", "PTPRCAP", "HLA-A", "LAG3", "LCK", "DUSP2", "APOBEC3G", "RP11-84E17.1"]
Treg_gene_list = ["CD3D", "CD3E", "IL32", "CD2", "HLA-A", "BATF", "CD27", "FOXP3", "TIGIT", "CORO1B", "CTLA4", "IL2RG", "CD3G", "TNFRSF18", "B2M", "LCK", "SIRPG", "GBP5", "SPOCK2", "HLA-B", "CARD16", "TNFRSF4", "SLAMF1", "HLA-C", "BTG1"]
Tgd_gene_list = ["CCL5", "CST7", "NKG7", "CD3D", "CD3E", "TRDV1", "GZMK", "DUSP2", "GZMA", "PRF1", "CTSW", "GNLY", "KLRG1", "TRGV9", "CD3G", "GZMH", "KLRD1", "TRDV2", "TRAC", "TRDC", "CCL4", "ZFP36L2", "MATK", "LAG3", "CD2"]
TMAIT_gene_list = ["CCL5", "CD3D", "CST7", "GZMA", "NKG7", "CD3E", "GZMK", "CD8A", "IL7R", "CD2", "PRF1", "CCL4", "CD3G", "DUSP2", "CD8B", "IL32", "IL2RG", "CTSW", "GZMH", "TRAV1-2", "LAG3", "CRIP1", "RP11-84E17.1", "ZFP36L2", "BTG1"]
NKT_gene_list = ["NKG7", "CST7", "KLRD1", "GNLY", "GZMA", "PRF1", "CTSW", "CD247", "GZMH", "KLRB1", "GZMM", "CD7", "HOPX", "FGFBP2", "CCL5", "CLIC3", "CCL4", "MATK", "SPON2", "KLRF1", "DUSP2", "XCL2", "IL2RB", "SYTL3", "SH2D1B"]
Th17_list = ["KLRB1", "RORA", "IL7R", "DONSON", "TRAC", "GPR183", "EVI2A", "EIF1", "ANKRD28", "GPR171", "BTG1", "STAT4", "MALAT1", "GPR65", "SNHG9", "CCL20", "TNFAIP3", "CD96", "PTGER4", "MYADM", "NAP1L1", "TANK", "CCR6", "FKBP11", "CD2"]

sc.tl.score_genes(adata, Lympho_gene_list, n_bins=30, score_name='score_Lympho')
sc.tl.score_genes(adata, Tnaive_gene_list, n_bins=30, score_name='score_Tnaive')
sc.tl.score_genes(adata, CD4_Ef_Mem_gene_list, n_bins=30, score_name='score_CD4Efmem')
sc.tl.score_genes(adata, CD8_Ef_Mem_gene_list, n_bins=30, score_name='score_CD8Efmem')
sc.tl.score_genes(adata, Treg_gene_list, n_bins=30, score_name='score_Treg')
sc.tl.score_genes(adata, Tgd_gene_list, n_bins=30, score_name='score_Tgd')
sc.tl.score_genes(adata, TMAIT_gene_list, n_bins=30, score_name='score_TMAIT')
sc.tl.score_genes(adata, NKT_gene_list, n_bins=30, score_name='score_NKT')
sc.tl.score_genes(adata, Th17_list, n_bins=30, score_name='score_Th17')

Lympho_with_high_score = adata.obs.loc[adata.obs['score_Lympho'] > 20, 'leiden'].unique()
Lympho_with_high_score

In [ ]:
Tn_with_high_score = adata.obs.loc[adata.obs['score_Tnaive'] > 20, 'leiden'].unique()
Tn_with_high_score

In [ ]:
CD4eff_with_high_score = adata.obs.loc[adata.obs['score_CD4Efmem'] > 20, 'leiden'].unique()
CD4eff_with_high_score

In [ ]:
CD8eff_with_high_score = adata.obs.loc[adata.obs['score_CD8Efmem'] > 20, 'leiden'].unique()
CD8eff_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_Tnaive > 1], basis='X_scVI_umap', color=['score_Lympho','score_Tnaive','score_CD4Efmem', 'score_CD8Efmem', 'score_Treg', 'score_Th17','score_Tgd', 'score_TMAIT', 'score_NKT', 'leiden', 'pct_counts_mt', "CD4", "CD8A", 'FOXP3'], ncols=2 )

In [ ]:
NK_gene_list = ["NKG7", "CST7", "KLRD1", "GNLY", "PRF1", "GZMA", "CCL5", "CCL4", "CD247", "FGFBP2", "CTSW", "KLRB1", "CLIC3", "SPON2", "GZMM", "KLRF1", "GZMH", "HOPX", "FCGR3A", "CD7", "MATK", "PYHIN1", "SH2D1B", "TXK", "TBX21"]
ILC_gene_list = ["NKG7", "KLRD1", "CST7", "GNLY", "PRF1", "GZMA", "CCL5", "CCL4", "KLRB1", "CTSW", "CD247", "FGFBP2", "CLIC3", "SPON2", "KLRF1", "GZMM", "HOPX", "GZMH", "CD7", "FCGR3A", "MATK", "PYHIN1", "SH2D1B", "TXK", "XCL2"]

sc.tl.score_genes(adata, NK_gene_list, n_bins=30, score_name='score_NK')
sc.tl.score_genes(adata, ILC_gene_list, n_bins=30, score_name='score_ILC')


NK_with_high_score = adata.obs.loc[adata.obs['score_NK'] > 20, 'leiden'].unique()
NK_with_high_score
ILC_with_high_score = adata.obs.loc[adata.obs['score_ILC'] > 20, 'leiden'].unique()
ILC_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_NK > 1], basis='X_scVI_umap', color=['score_NK','score_ILC', 'leiden', 'pct_counts_mt'], ncols=2, legend_loc= 'on data', color_map = "viridis")

In [ ]:
Bcell_gene_list = ["MS4A1", "CD79A", "BANK1", "CD79B", "CD37", "RALGPS2", "IGHM", "IGKC", "LINC00926", "BLK", "VPREB3", "HVCN1", "NIBAN3", "IRF8", "AFF3", "FCRL1", "FCRLA", "PRKCB", "SMIM14", "RIPOR2", "HLA-DRA", "IGHA1"]
Plasma_gene_list = ["MZB1", "IGKC", "TXNDC5", "XBP1", "IGHG1", "DERL3", "JCHAIN", "FKBP11", "IGHG3", "IGHA1", "SEC11C", "GLCCI1", "IFNG-AS1", "IGHG4", "SEL1L3", "TXNDC11", "CREB3L2", "FUT8", "IGLC2", "TENT5C", "FCRL5", "DENND5B", "POU2AF1", "SPATS2", "GNG7"]

sc.tl.score_genes(adata, Bcell_gene_list, n_bins=30, score_name='score_Bcell')
sc.tl.score_genes(adata, Plasma_gene_list, n_bins=30, score_name='score_Plasmacell')

Bcell_with_high_score = adata.obs.loc[adata.obs['score_Bcell'] > 20, 'leiden'].unique()
Bcell_with_high_score
Plasmacell_with_high_score = adata.obs.loc[adata.obs['score_Plasmacell'] > 20, 'leiden'].unique()
Plasmacell_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_Plasmacell > 1], basis='X_scVI_umap', color=['score_Bcell', 'score_Plasmacell', 'leiden', 'pct_counts_mt'], ncols=2, legend_loc= 'on data', color_map = "viridis")

## Neuron

In [ ]:
neuro_genelist = ["NRXN1", "NRXN3", "ADGRB3", "CADM2", "XKR4", "RBMS3", "FRMD4A", "GRIK2", "NEGR1", "CADM1", "NCAM1", "SORCS1", "SLC35F1", "CDH19", "PCDH9", "NKAIN3", "EHBP1", "ZNF536", "CASC15", "RP11-141M1.3", "NLGN4X", "SCN7A", "NCAM2", "TENM3", "DLC1"]

sc.tl.score_genes(adata, neuro_genelist, n_bins=30, score_name='score_neuro')

Neuro_with_high_score = adata.obs.loc[adata.obs['score_neuro'] > 10, 'leiden'].unique()
Neuro_with_high_score

In [ ]:
sc.pl.embedding(adata[adata.obs.score_neuro > 15], basis='X_scVI_umap', color=['score_neuro', 'leiden', 'pct_counts_mt'], ncols=2, legend_loc= 'on data')

## Unknown clusters manual annotation

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden', method='logreg')

In [ ]:
top_genes = adata.uns['rank_genes_groups']['names']['?1']
top_genes[:50]

In [ ]:
genemarker_dict = {'3':'HighMT_PTS1/2',
                   '1':'PTS1',
                   '33':'HighMT_PTS1|EC',
                   '12':'PTS3',
                   '43':'POD',
                   '9':'LowQ_PEC|VSMC|Neuro|Immune',
                   '41':'PEC',
                   '0':'HighMT_PT/MD', #some cells are MD not the whole cluster
                   '27':'DCT',
                   '34':'PapE',
                   '17':'Doub_CNT-IC-A|CD-ICA|CNT-PC', #part of 17 belongs to CD-ICA and B
                   '11':'CD-PC',
                   '38':'Doub_CD-ICA|CD-ICB',
                   '10':'TAL', 
                   '18':'HighMT_TAL|EC', #there is part that might be PTS2
                   '36':'LowQ_DTL|DC|Tcell',
                   '8':'HighMT_Doub_ATL|DTL',
                   '21':'ATL',
                   '40':'LowQ_PT|EC',
                   '6':'HighMT_PTS1/2',
                   '5':'HighMT_LowQ_PT',
                   '44':'HighMT_LowQ_PT|B cells',
                   '39':'HighMT_LowQ_PT',
                   '52':'HighMT_PT_IL32',
                   '23':'Doub_EC-AEA|EC-DVR',
                   '2':'EC-AVR',
                   '22':'Doub_EC-PTC|MC',
                   '50':'EC-LYM',
                   '7':'HighMT_Doub_Macro|DC', #check subcluster of 7 for highMT and split DC out
                   '49':'HighMT_PT|Macro',
                   '42':'Macro',
                   '19':'Monocytes', #need to split to CD16expressing
                   '4':'CD4eff_mem|Treg',
                   '15':'MAIT|Tgd||NKT',
                   '20':'CD8_eff_mem', 
                   '30':'CD8|CD4_naive',
                   '16':'NK|ILC',
                   '45':'Activated_Mast cells',
                   '28':'B cells',
                   '36':'Plasma cells',
                   '13':'PTS3',
                   '14':'CNT-PC',
                   '24':'Doub_PTS1/2|Fibro',
                   '26':'Mast cells',
                   '2':'Fibroblasts',
                   '29':'Damaged_urothelial',
                   '31':'HighMT_Neural cells',
                   '32':'EC-AVR_damaged',
                   '35':'SULF1+_EC',
                   '37':'PEC|Pancreatic',
                   '46':'Doub_PEC|PT',
                   '47':'IC-A',
                   '48':'LowQ_Monocytes',
                   '51':'Neutrophil',
                   '53':'LowQ_PT',
                   '54':'LowQ',
                   '55':'Activated_CD8',
                   '25':'DTL_ATL'
                  }
adata.obs['annotation_genemarker'] = adata.obs['leiden'].map(genemarker_dict)

In [ ]:
# Get unique clusters
clusters = adata.obs['annotation_genemarker'].unique()

# Initialize a new column for hierarchical subclustering results
adata.obs['leiden_R'] = None

# Iterate over each cluster
for cluster in clusters:
    # Subset the data for the current cluster
    cluster_mask = adata.obs['annotation_genemarker'] == cluster
    cluster_data = adata[cluster_mask]
    
    # Perform Leiden clustering on the subset
    sc.tl.leiden(cluster_data, resolution=0.3, key_added='leiden_subcluster')
    
    # Create hierarchical labels (e.g., Cluster1_1, Cluster1_2, etc.)
    subcluster_labels = [f"{cluster}_{subcluster}" for subcluster in cluster_data.obs['leiden_subcluster']]
    
    # Add the hierarchical labels to the original adata object
    adata.obs.loc[cluster_data.obs.index, 'leiden_R'] = subcluster_labels

In [ ]:
sc.pl.embedding(adata, basis='X_scVI_umap', color=['leiden'], ncols=1, legend_loc='on data', legend_fontsize='5')#[adata.obs['score_aPT']>1.6]

In [ ]:
sc.pl.embedding(adata, basis='X_scVI_umap', color=['leiden_R'], ncols=1, palette=list(matplotlib.colors.CSS4_COLORS.values()))#[adata.obs_names.isin(obs_names_cleaned)]

In [ ]:
Scanpyplus.DEmarkers(adata, 'Plasma cells_0', 'Plasma cells_1', 'leiden_R', obslist=['leiden', 'pct_counts_mt'])

In [ ]:
l_R_anno = {
            'B cells_0': 'Bcells_activated',
            'B cells_1': 'Bcells_naive',
            'B cells_2': 'Bcells_activated',
            'B cells_3': 'Bcells_activated',
            'B cells_4': 'Bcells_naive',
            'B cells_5': 'Bcells_activated',
            'B cells_6': 'Doub_Tcell|Bcell',
            'B cells_7': 'Bcells_memory',
            'B cells_8': 'Doube_Tcell|Bcell',
            'B cells_9': 'LowQ_Bcells',
            'B cells_10': 'LowQ_Bcells',
            'B cells_11': 'LowQ_Bcells',
            'B cells_12': 'LowQ_Bcells',
            'Plasma cells_0': 'Plasma cells',
            'Plasma cells_1': 'Plasma cells',
            'Plasma cells_2': 'Plasma cells',
            'Plasma cells_3': 'Plasma cells',
            'Plasma cells_4': 'Plasma cells',
            'Plasma cells_5': 'LowQ_Plasma cells',
            'Plasma cells_6': 'Plasma cells',
            'Plasma cells_7': 'Plasma cells',
            'Plasma cells_8': 'Plasma cells',
            'Plasma cells_9': 'Plasma cells',
            'Plasma cells_10': 'Plasma cells',
            'Plasma cells_11': 'Plasma cells',
            'Doub_EC-AEA|EC-DVR_0': 'EC-DVR',
            'Doub_EC-AEA|EC-DVR_1': 'EC-AEA',
            'Doub_EC-AEA|EC-DVR_2': 'EC-DVR',
            'Doub_EC-AEA|EC-DVR_3': 'EC-AEA_SULF1+',
            'Doub_EC-AEA|EC-DVR_4': 'EC-DVR',
            'Doub_EC-AEA|EC-DVR_5': 'EC-AVR',
            'Doub_EC-AEA|EC-DVR_6': 'EC-DVR',
            'Doub_EC-PTC|MC_0': 'EC-cap',
            'Doub_EC-PTC|MC_1': 'EC-Glomerular_cap',
            'Doub_EC-PTC|MC_2': 'EC-Glomerular_cap',
            'Doub_EC-PTC|MC_3': 'dEC-cap',
            'Doub_EC-PTC|MC_4': 'EC-Glomerular_cap',
            'Doub_EC-PTC|MC_5': 'dEC-PTC',
            'EC-AVR_damaged_0': 'Doub_EC|Macro',
            'EC-AVR_damaged_1': 'Doub_EC|Macro',
            'EC-AVR_damaged_2': 'Doub_EC|Macro',
            'EC-AVR_damaged_3': 'Doub_EC|Macro',
            'EC-AVR_damaged_4': 'EC-AVR',
            'EC-AVR_damaged_5': 'EC-DVR',
            'EC-AVR_damaged_6': 'Doub_EC|Macro',
            'EC-AVR_damaged_7': 'Doub_EC|Macro',
            'EC-AVR_damaged_8': 'Doub_EC|Macro',
            'EC-LYM_0': 'EC-LYM',
            'EC-LYM_1': 'EC-LYM',
            'EC-LYM_2': 'EC-LYM',
            'EC-LYM_3': 'dEC-LYM',
            'Fibroblasts_0': 'EC-PTC',
            'Fibroblasts_1': 'Doub_EC-PTC|AVR',
            'Fibroblasts_2': 'EC-AVR',
            'Fibroblasts_3': 'cycEC-PTC',
            'Fibroblasts_4': 'dEC-cap',
            'Fibroblasts_5': 'EC-venus',
            'Fibroblasts_6': 'EC-PTC',
            'SULF1+_EC_0': 'EC-cap_STC1+',
            'SULF1+_EC_1': 'EC-cap_STC1+',
            'SULF1+_EC_2': 'EC-cap_STC1+',
            'SULF1+_EC_3': 'Doub_EC|PC',
            'SULF1+_EC_4': 'EC-cap_STC1+',
            'SULF1+_EC_5': 'EC-cap_STC1+',
            'SULF1+_EC_6': 'Doub_EC|PC',
            'SULF1+_EC_7': 'EC-cap_STC1+',
            'SULF1+_EC_8': 'EC-cap_STC1+',
            'SULF1+_EC_9': 'EC-cap_STC1+',
            'LowQ_PEC|VSMC|Neuro|Immune_0': 'VSCM',
            'LowQ_PEC|VSMC|Neuro|Immune_1': 'Pericytes',
            'LowQ_PEC|VSMC|Neuro|Immune_2': 'MC',
            'LowQ_PEC|VSMC|Neuro|Immune_3': 'Fibroblasts_NEGR1+',
            'LowQ_PEC|VSMC|Neuro|Immune_4': 'Fibroblasts',
            'LowQ_PEC|VSMC|Neuro|Immune_5': 'EC-DVR',
            'LowQ_PEC|VSMC|Neuro|Immune_6': 'Doub_Fibroblasts|Immune',
            'LowQ_PEC|VSMC|Neuro|Immune_7': 'MC',
            'LowQ_PEC|VSMC|Neuro|Immune_8': 'CD-PC',
            'LowQ_PEC|VSMC|Neuro|Immune_9': 'Doub_VSMC|Immune',
            'LowQ_PEC|VSMC|Neuro|Immune_10': 'VSMC',
            'CD8_eff_mem_0': 'CD8_activated_exhausted',
            'CD8_eff_mem_1': 'LowQ_CD8',
            'CD8_eff_mem_2': 'CD8_cycling',
            'CD8_eff_mem_3': 'CD8_memory_resident',
            'CD8_eff_mem_4': 'LowQ_CD8',
            'CD8_eff_mem_5': 'CD8_cycling',
            'CD8|CD4_naive_0': 'CD4_naive',
            'CD8|CD4_naive_1': 'CD8_naive',
            'CD8|CD4_naive_2': 'CD8_naive',
            'CD8|CD4_naive_3': 'CD8_naive',
            'CD8|CD4_naive_4': 'CD8_naive',
            'CD8|CD4_naive_5': 'CD8_naive',
            'CD8|CD4_naive_6': 'CD4_naive',
            'CD8|CD4_naive_7': 'CD8_naive',
            'CD8|CD4_naive_8': 'CD8_naive',
            'CD4eff_mem|Treg_0': 'CD8_memory_resident',
            'CD4eff_mem|Treg_1': 'CD4_memory_resident',
            'CD4eff_mem|Treg_2': 'Tregs',
            'CD4eff_mem|Treg_3': 'CD4_memory_resident',
            'CD4eff_mem|Treg_4': 'CD4_central_memory',
            'CD4eff_mem|Treg_5': 'LowQ_CD8_memory',
            'CD4eff_mem|Treg_6': 'CD8_activated_cytotoxic',
            'CD4eff_mem|Treg_7': 'gdT cells',
            'CD4eff_mem|Treg_8': 'LowQ_Immune',
            'CD4eff_mem|Treg_9': 'LowQ_Immune',
            'CD4eff_mem|Treg_10': 'LowQ_Immune',
            'CD4eff_mem|Treg_11': 'LowQ_Immune',
            'CD4eff_mem|Treg_12': 'LowQ_Immune',
            'CD4eff_mem|Treg_13': 'LowQ_Immune',
            'CD4eff_mem|Treg_14': 'LowQ_Immune',            
            'CD4eff_mem|Treg_15':'LowQ_Immune',
            'CD4eff_mem|Treg_16':'LowQ_Immune',
            'CD4eff_mem|Treg_17':'LowQ_Immune',
            'CD4eff_mem|Treg_18':'LowQ_Immune',
            'CD4eff_mem|Treg_19':'LowQ_Immune',
            'CD4eff_mem|Treg_20':'LowQ_Immune',
            'CD4eff_mem|Treg_21':'LowQ_Immune',
            'CD4eff_mem|Treg_22':'LowQ_Immune',
            'CD4eff_mem|Treg_23':'LowQ_Immune',
            'CD4eff_mem|Treg_24':'LowQ_Immune',
            'CD4eff_mem|Treg_25':'LowQ_Immune',
            'CD4eff_mem|Treg_26':'LowQ_Immune',
            'CD4eff_mem|Treg_27':'LowQ_Immune',
            'CD4eff_mem|Treg_28':'LowQ_Immune',
            'CD4eff_mem|Treg_29':'LowQ_Immune',
            'CD4eff_mem|Treg_30':'LowQ_Immune',
            'CD4eff_mem|Treg_31':'LowQ_Immune',
            'CD4eff_mem|Treg_32':'LowQ_Immune',
            'CD4eff_mem|Treg_33':'LowQ_Immune',
            'CD4eff_mem|Treg_34':'LowQ_Immune',
            'CD4eff_mem|Treg_35':'LowQ_Immune',
            'CD4eff_mem|Treg_36':'LowQ_Immune',
            'CD4eff_mem|Treg_37':'LowQ_Immune',
            'CD4eff_mem|Treg_38':'LowQ_Immune',
            'CD4eff_mem|Treg_39':'LowQ_Immune',
            'CD4eff_mem|Treg_40':'LowQ_Immune',
            'CD4eff_mem|Treg_41':'LowQ_Immune',
            'CD4eff_mem|Treg_42':'LowQ_Immune',
            'CD4eff_mem|Treg_43':'LowQ_Immune',
            'CD4eff_mem|Treg_44':'LowQ_Immune',
            'CD4eff_mem|Treg_45':'LowQ_Immune',
            'CD4eff_mem|Treg_46':'LowQ_Immune',
            'CD4eff_mem|Treg_47':'LowQ_Immune',
            'CD4eff_mem|Treg_48':'LowQ_Immune',
            'CD4eff_mem|Treg_49':'LowQ_Immune',
            'CD4eff_mem|Treg_50':'LowQ_Immune',
            'CD4eff_mem|Treg_51':'LowQ_Immune',
            'CD4eff_mem|Treg_52':'LowQ_Immune',
            'CD4eff_mem|Treg_53':'LowQ_Immune',
            'CD4eff_mem|Treg_54':'LowQ_Immune',
            'CD4eff_mem|Treg_55':'LowQ_Immune',
            'CD4eff_mem|Treg_56':'LowQ_Immune',
            'CD4eff_mem|Treg_57':'LowQ_Immune',
            'CD4eff_mem|Treg_58':'LowQ_Immune',
            'CD4eff_mem|Treg_59':'LowQ_Immune',
            'CD4eff_mem|Treg_60':'LowQ_Immune',
            'CD4eff_mem|Treg_61':'LowQ_Immune',
            'CD4eff_mem|Treg_62':'LowQ_Immune',
            'CD4eff_mem|Treg_63':'LowQ_Immune',
            'CD4eff_mem|Treg_64':'LowQ_Immune',
            'CD4eff_mem|Treg_65':'LowQ_Immune',
            'CD4eff_mem|Treg_66':'LowQ_Immune',
            'CD4eff_mem|Treg_67':'LowQ_Immune',
            'CD4eff_mem|Treg_68':'LowQ_Immune',
            'CD4eff_mem|Treg_69':'LowQ_Immune',
            'CD4eff_mem|Treg_70':'LowQ_Immune',
            'CD4eff_mem|Treg_71':'LowQ_Immune',
            'CD4eff_mem|Treg_72':'LowQ_Immune',
            'CD4eff_mem|Treg_73':'LowQ_Immune',
            'CD4eff_mem|Treg_74':'LowQ_Immune',
            'CD4eff_mem|Treg_75':'LowQ_Immune',
            'CD4eff_mem|Treg_76':'LowQ_Immune',
            'CD4eff_mem|Treg_77':'LowQ_Immune',
            'CD4eff_mem|Treg_78':'LowQ_Immune',
            'CD4eff_mem|Treg_79':'LowQ_Immune',
            'CD4eff_mem|Treg_80':'LowQ_Immune',
            'CD4eff_mem|Treg_81':'LowQ_Immune',
            'CD4eff_mem|Treg_82':'LowQ_Immune',
            'CD4eff_mem|Treg_83':'LowQ_Immune',
            'CD4eff_mem|Treg_84':'LowQ_Immune',
            'CD4eff_mem|Treg_85':'LowQ_Immune',
            'CD4eff_mem|Treg_86':'LowQ_Immune',
            'CD4eff_mem|Treg_87':'LowQ_Immune',
            'CD4eff_mem|Treg_88':'LowQ_Immune',
            'CD4eff_mem|Treg_89':'LowQ_Immune',
            'CD4eff_mem|Treg_90':'LowQ_Immune',
            'CD4eff_mem|Treg_91':'LowQ_Immune',
            'CD4eff_mem|Treg_92':'LowQ_Immune',
            'CD4eff_mem|Treg_93':'LowQ_Immune',
            'Neutrophil_0': 'Neutrophils',
            'Neutrophil_1': 'Neutrophils',
            'Neutrophil_2': 'Neutrophils_activated',
            'Neutrophil_3': 'Neutrophils_activated',
            'Neutrophil_4': 'Neutrophils',
            'Neutrophil_5': 'Neutrophils_activated',
            'Neutrophil_6': 'Neutrophils_activated',
            'Neutrophil_7': 'Neutrophils_activated',
            'NK|ILC_0': 'CD8_effector_cytoxic',
            'NK|ILC_1': 'Doub_PT|NK',
            'NK|ILC_2': 'CD8_effector_cytoxic',
            'NK|ILC_3': 'Doub_Tcell|Bcell',
            'NK|ILC_4': 'CD8_effector_cytoxic',
            'NK|ILC_5': 'CD8_effector_cytoxic',
            'NK|ILC_6': 'NK_CD56dim_CD16+',
            'NK|ILC_7': 'CD8_effector_cytoxic',
            'NK|ILC_8': 'CD8_effector_cytoxic',
            'MAIT|Tgd||NKT_0': 'CD8_effector_cytoxic',
            'MAIT|Tgd||NKT_1': 'NKT',
            'MAIT|Tgd||NKT_2': 'CD8_effector_cytoxic',
            'MAIT|Tgd||NKT_3': 'NK_CD56bright_CD16-',
            'MAIT|Tgd||NKT_4': 'CD4_resident_memory',
            'MAIT|Tgd||NKT_5': 'CD8_effector_cytoxic',
            'MAIT|Tgd||NKT_6': 'NK_CD56bright_CD16+',
            'MAIT|Tgd||NKT_7': 'MAIT cells',
            'MAIT|Tgd||NKT_8': 'LowQ_Immune',
            'MAIT|Tgd||NKT_9':'LowQ_Immune',
            'MAIT|Tgd||NKT_10':'LowQ_Immune',
            'MAIT|Tgd||NKT_11':'LowQ_Immune',
            'MAIT|Tgd||NKT_12':'LowQ_Immune',
            'MAIT|Tgd||NKT_13':'LowQ_Immune',
            'MAIT|Tgd||NKT_14':'LowQ_Immune',
            'MAIT|Tgd||NKT_15':'LowQ_Immune',
            'MAIT|Tgd||NKT_16':'LowQ_Immune',
            'MAIT|Tgd||NKT_17':'LowQ_Immune',
            'MAIT|Tgd||NKT_18':'LowQ_Immune',
            'MAIT|Tgd||NKT_19':'LowQ_Immune',
            'MAIT|Tgd||NKT_20':'LowQ_Immune',
            'MAIT|Tgd||NKT_21':'LowQ_Immune',
            'MAIT|Tgd||NKT_22':'LowQ_Immune',
            'MAIT|Tgd||NKT_23':'LowQ_Immune',
            'MAIT|Tgd||NKT_24':'LowQ_Immune',
            'MAIT|Tgd||NKT_25':'LowQ_Immune',
            'MAIT|Tgd||NKT_26':'LowQ_Immune',
            'MAIT|Tgd||NKT_27':'LowQ_Immune',
            'MAIT|Tgd||NKT_28':'LowQ_Immune',
            'MAIT|Tgd||NKT_29':'LowQ_Immune',
            'MAIT|Tgd||NKT_30':'LowQ_Immune',
            'MAIT|Tgd||NKT_31':'LowQ_Immune',
            'MAIT|Tgd||NKT_32':'LowQ_Immune',
            'MAIT|Tgd||NKT_33':'LowQ_Immune',
            'MAIT|Tgd||NKT_34':'LowQ_Immune',
            'MAIT|Tgd||NKT_35':'LowQ_Immune',
            'MAIT|Tgd||NKT_36':'LowQ_Immune',
            'MAIT|Tgd||NKT_37':'LowQ_Immune',
            'MAIT|Tgd||NKT_38':'LowQ_Immune',
            'MAIT|Tgd||NKT_39':'LowQ_Immune',
            'MAIT|Tgd||NKT_40':'LowQ_Immune',
            'MAIT|Tgd||NKT_41':'LowQ_Immune',
            'MAIT|Tgd||NKT_42':'LowQ_Immune',
            'MAIT|Tgd||NKT_43':'LowQ_Immune',
            'MAIT|Tgd||NKT_44':'LowQ_Immune',
            'MAIT|Tgd||NKT_45':'LowQ_Immune',
            'MAIT|Tgd||NKT_46':'LowQ_Immune',
            'MAIT|Tgd||NKT_47':'LowQ_Immune',
            'MAIT|Tgd||NKT_48':'LowQ_Immune',
            'MAIT|Tgd||NKT_49':'LowQ_Immune',
            'MAIT|Tgd||NKT_50':'LowQ_Immune',
            'MAIT|Tgd||NKT_51':'LowQ_Immune',
            'MAIT|Tgd||NKT_52':'LowQ_Immune',
            'MAIT|Tgd||NKT_53':'LowQ_Immune',
            'MAIT|Tgd||NKT_54':'LowQ_Immune',
            'MAIT|Tgd||NKT_55':'LowQ_Immune',
            'MAIT|Tgd||NKT_56':'LowQ_Immune',
            'MAIT|Tgd||NKT_57':'LowQ_Immune',
            'MAIT|Tgd||NKT_58':'LowQ_Immune',
            'MAIT|Tgd||NKT_59':'LowQ_Immune',
            'MAIT|Tgd||NKT_60':'LowQ_Immune',
            'MAIT|Tgd||NKT_61':'LowQ_Immune',
            'MAIT|Tgd||NKT_62':'LowQ_Immune',
            'MAIT|Tgd||NKT_63':'LowQ_Immune',
            'MAIT|Tgd||NKT_64':'LowQ_Immune',
            'MAIT|Tgd||NKT_65':'LowQ_Immune',
            'MAIT|Tgd||NKT_66':'LowQ_Immune',
            'MAIT|Tgd||NKT_67':'LowQ_Immune',
            'MAIT|Tgd||NKT_68':'LowQ_Immune',
            'MAIT|Tgd||NKT_69':'LowQ_Immune',
            'MAIT|Tgd||NKT_70':'LowQ_Immune',
            'MAIT|Tgd||NKT_71':'LowQ_Immune',
            'MAIT|Tgd||NKT_72':'LowQ_Immune',
            'MAIT|Tgd||NKT_73':'LowQ_Immune',
            'MAIT|Tgd||NKT_74':'LowQ_Immune',
            'MAIT|Tgd||NKT_75':'LowQ_Immune',
            'MAIT|Tgd||NKT_76':'LowQ_Immune',
            'MAIT|Tgd||NKT_77':'LowQ_Immune',
            'MAIT|Tgd||NKT_78':'LowQ_Immune',
            'MAIT|Tgd||NKT_79':'LowQ_Immune',
            'MAIT|Tgd||NKT_80':'LowQ_Immune',
            'MAIT|Tgd||NKT_81':'LowQ_Immune',
            'MAIT|Tgd||NKT_82':'LowQ_Immune',
            'MAIT|Tgd||NKT_83':'LowQ_Immune',
            'MAIT|Tgd||NKT_84':'LowQ_Immune',
            'MAIT|Tgd||NKT_85':'LowQ_Immune',
            'MAIT|Tgd||NKT_86':'LowQ_Immune',
            'MAIT|Tgd||NKT_87':'LowQ_Immune',
            'MAIT|Tgd||NKT_88':'LowQ_Immune',
            'MAIT|Tgd||NKT_89':'LowQ_Immune',
            'MAIT|Tgd||NKT_90':'LowQ_Immune',
            'MAIT|Tgd||NKT_91':'LowQ_Immune',
            'MAIT|Tgd||NKT_92':'LowQ_Immune',
            'HighMT_Doub_Macro|DC_0': 'Macrophages_TREM2+',
            'HighMT_Doub_Macro|DC_1': 'Macrophages_M2_MRC1+',
            'HighMT_Doub_Macro|DC_2': 'cDC2',
            'HighMT_Doub_Macro|DC_3': 'Macrophages_Resident',
            'HighMT_Doub_Macro|DC_4': 'Macrophages_TREM2+',
            'HighMT_Doub_Macro|DC_5': 'Monocyte-Macrophages',
            'HighMT_Doub_Macro|DC_6': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_7': 'Macrophages_M2',
            'HighMT_Doub_Macro|DC_8': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_9': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_10': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_11': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_12': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_13': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_14': 'LowQ_Macrophages',
            'HighMT_Doub_Macro|DC_15': 'Macrophages_TREM2+',
            'LowQ_Monocytes_0':'PT_Zhang',
            'LowQ_Monocytes_1':'PT_Zhang',
            'LowQ_Monocytes_2':'PT_Zhang',
            'LowQ_Monocytes_3':'PT_Zhang',
            'LowQ_Monocytes_4':'PT_Zhang',
            'LowQ_Monocytes_5':'PT_Zhang',
            'LowQ_Monocytes_6':'PT_Zhang',
            'Macro_0': 'Doub_Macro|PT',
            'Macro_1': 'Doub_Macro|PT',
            'Macro_2': 'Doub_Macro|PT',
            'Macro_3': 'Doub_Macro|PT',
            'Macro_4': 'Doub_Macro|PT',
            'Macro_5': 'Doub_Macro|PT',
            'Macro_6': 'Doub_Macro|PT',
            'Macro_7': 'Doub_Macro|PT',
            'Macro_8': 'Doub_Macro|PT',
            'Macro_9': 'Doub_Macro|PT',
            'Monocytes_0': 'Monocytes non-classical_CD16+',
            'Monocytes_1': 'Monocytes classical_CD14+',
            'Monocytes_2': 'Monocytes non-classical_CD16+',
            'Monocytes_3': 'Monocytes classical_activated',
            'Monocytes_4': 'Monocytes classical_migratory',
            'Monocytes_5': 'Monocytes non-classical_activated',
            'Monocytes_6': 'LowQ_Monocyte',
            'Monocytes_7': 'LowQ_Monocyte',
            'Monocytes_8': 'LowQ_Monocyte',
            'Monocytes_9': 'LowQ_Monocyte',
            'Monocytes_10': 'LowQ_Monocyte',
            'Monocytes_11': 'LowQ_Monocyte',
            'Monocytes_12': 'LowQ_Monocyte',
            'Monocytes_13': 'LowQ_Monocyte',
            'Monocytes_14': 'LowQ_Monocyte',
            'Monocytes_15': 'LowQ_Monocyte',
            'Monocytes_16': 'LowQ_Monocyte',
            'Monocytes_17': 'LowQ_Monocyte',
            'Monocytes_18': 'LowQ_Monocyte',
            'Monocytes_19': 'LowQ_Monocyte',
            'Monocytes_20': 'LowQ_Monocyte',
            'Monocytes_21': 'LowQ_Monocyte',
            'Monocytes_22': 'LowQ_Monocyte',
            'Monocytes_23': 'LowQ_Monocyte',
            'Monocytes_24': 'LowQ_Monocyte',
            'Monocytes_25': 'LowQ_Monocyte',
            'Monocytes_26': 'LowQ_Monocyte',
            'Monocytes_27': 'LowQ_Monocyte',
            'Monocytes_28': 'LowQ_Monocyte',
            'Monocytes_29': 'LowQ_Monocyte',
            'Monocytes_30': 'LowQ_Monocyte',
            'Monocytes_31': 'LowQ_Monocyte',
            'Monocytes_32': 'LowQ_Monocyte',
            'Monocytes_33': 'LowQ_Monocyte',
            'Monocytes_34': 'LowQ_Monocyte',
            'Monocytes_35': 'LowQ_Monocyte',
            'Monocytes_36': 'LowQ_Monocyte',
            'Monocytes_37': 'LowQ_Monocyte',
            'Monocytes_38': 'LowQ_Monocyte',
            'Monocytes_39': 'LowQ_Monocyte',
            'Monocytes_40': 'LowQ_Monocyte',
            'Monocytes_41': 'LowQ_Monocyte',
            'Monocytes_42': 'LowQ_Monocyte',
            'Monocytes_43': 'LowQ_Monocyte',
            'Monocytes_44': 'LowQ_Monocyte',
            'Monocytes_45': 'LowQ_Monocyte',
            'Monocytes_46': 'LowQ_Monocyte',
            'Monocytes_47': 'LowQ_Monocyte',
            'Monocytes_48': 'LowQ_Monocyte',
            'Monocytes_49': 'LowQ_Monocyte',
            'Monocytes_50': 'LowQ_Monocyte',
            'Monocytes_51': 'LowQ_Monocyte',
            'Monocytes_52': 'LowQ_Monocyte',
            'Monocytes_53': 'LowQ_Monocyte',
            'Monocytes_54': 'LowQ_Monocyte',
            'Monocytes_55': 'LowQ_Monocyte',
            'Monocytes_56': 'LowQ_Monocyte',
            'Monocytes_57': 'LowQ_Monocyte',
            'Monocytes_58': 'LowQ_Monocyte',
            'Monocytes_59': 'LowQ_Monocyte',
            'Monocytes_60': 'LowQ_Monocyte',
            'Monocytes_61': 'LowQ_Monocyte',
            'Monocytes_62': 'LowQ_Monocyte',
            'Monocytes_63': 'LowQ_Monocyte',
            'Monocytes_64': 'LowQ_Monocyte',
            'Monocytes_65': 'LowQ_Monocyte',
            'Monocytes_66': 'LowQ_Monocyte',
            'Monocytes_67': 'LowQ_Monocyte',
            'Monocytes_68': 'LowQ_Monocyte',
            'Monocytes_69': 'LowQ_Monocyte',
            'Monocytes_70': 'LowQ_Monocyte',
            'Monocytes_71': 'LowQ_Monocyte',
            'Monocytes_72': 'LowQ_Monocyte',
            'Monocytes_73': 'LowQ_Monocyte',
            'Mast cells_0': 'CD8_effector_cytoxic',
            'Mast cells_1': 'Doub_Tcell|Macro',
            'Mast cells_2': 'Doub_Tcell|Macro',
            'Mast cells_3': 'Monocytes classical_CD14+',
            'Mast cells_4': 'pDCs',
            'Mast cells_5': 'LowQ_Monocyte',
            'Mast cells_6': 'Doub_Tcell|Macro',
            'Mast cells_7': 'Monocytes non-classical_CD16+',
            'Mast cells_8': 'cDCs',
            'Mast cells_9': 'CD8_effector_cytoxic',
            'Mast cells_10': 'LowQ_Immune',
            'Mast cells_11': 'LowQ_Immune',
            'Mast cells_12': 'LowQ_Immune',
            'Mast cells_13': 'LowQ_Immune',
            'Mast cells_14': 'LowQ_Immune',
            'Mast cells_15': 'LowQ_Immune',
            'Mast cells_16': 'LowQ_Immune',
            'Mast cells_17': 'LowQ_Immune',
            'Mast cells_18': 'LowQ_Immune',
            'Mast cells_19': 'LowQ_Immune',
            'Mast cells_20': 'LowQ_Immune',
            'Mast cells_21': 'LowQ_Immune',
            'Mast cells_22': 'LowQ_Immune',
            'Mast cells_23': 'LowQ_Immune',
            'Mast cells_24': 'LowQ_Immune',
            'Mast cells_25': 'LowQ_Immune',
            'Mast cells_26': 'LowQ_Immune',
            'Mast cells_27': 'LowQ_Immune',
            'Mast cells_28': 'LowQ_Immune',
            'Mast cells_29': 'LowQ_Immune',
            'Mast cells_30': 'LowQ_Immune',
            'Mast cells_31': 'LowQ_Immune',
            'Mast cells_32': 'LowQ_Immune',
            'Mast cells_33': 'LowQ_Immune',
            'Mast cells_34': 'LowQ_Immune',
            'Mast cells_35': 'LowQ_Immune',
            'Mast cells_36': 'LowQ_Immune',
            'Mast cells_37': 'LowQ_Immune',
            'Mast cells_38': 'LowQ_Immune',
            'Mast cells_39': 'LowQ_Immune',
            'Mast cells_40': 'LowQ_Immune',
            'Mast cells_41': 'LowQ_Immune',
            'Mast cells_42': 'LowQ_Immune',
            'Mast cells_43': 'LowQ_Immune',
            'Mast cells_44': 'LowQ_Immune',
            'Mast cells_45': 'LowQ_Immune',
            'Mast cells_46': 'LowQ_Immune',
            'Mast cells_47': 'LowQ_Immune',
            'Mast cells_48': 'LowQ_Immune',
            'Mast cells_49': 'LowQ_Immune',
            'Mast cells_50': 'LowQ_Immune',
            'Mast cells_51': 'LowQ_Immune',
            'Mast cells_52': 'LowQ_Immune',
            'Mast cells_53': 'LowQ_Immune',
            'Mast cells_54': 'LowQ_Immune',
            'Mast cells_55': 'LowQ_Immune',
            'Mast cells_56': 'LowQ_Immune',
            'Mast cells_57': 'LowQ_Immune',
            'Mast cells_58': 'LowQ_Immune',
            'Activated_Mast cells_0': 'Mast cells_activated',
            'Activated_Mast cells_1': 'Mast cells',
            'Activated_Mast cells_2': 'Mast cells',
            'Activated_Mast cells_3': 'Mast cells_activated',
            'Activated_Mast cells_4': 'Doub_Mast|Monocytes',
            'Activated_Mast cells_5': 'B cells_activated',
            'Activated_Mast cells_6': 'LowQ',
            'Activated_Mast cells_7': 'LowQ',
            'Activated_Mast cells_8': 'CD8 effector_cytoxic',
            'Activated_Mast cells_9': 'LowQ_Tcell',
            'Activated_Mast cells_10': 'LowQ_Tcell',
            'Activated_Mast cells_11': 'LowQ_Tcell',
            'Activated_CD8_0': 'CD8_effector_cycling',
            'ATL_0': 'ATL',
            'ATL_1': 'aTAL',
            'ATL_2': 'aATL_CCL2+',
            'ATL_3': 'Doub_ATL|Tcell',
            'ATL_4': 'ATL',
            'ATL_5': 'MD',
            'ATL_6': 'Doub_PT|ATL',
            'ATL_7': 'ATL',
            'DTL_ATL_0': 'DTL1',
            'DTL_ATL_1': 'dDTL1/2',
            'DTL_ATL_2': 'Doub_DTL|PT',
            'DTL_ATL_3': 'ATL',
            'DTL_ATL_4': 'dDTL2',
            'DTL_ATL_5': 'aPT',
            'DTL_ATL_6': 'ATL_SERPINA1+',
            'DTL_ATL_7': 'aTAL',
            'DTL_ATL_8': 'Doub_DTL|Tcell',
            'HighMT_Doub_ATL|DTL_0': 'aTAL',
            'HighMT_Doub_ATL|DTL_1': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_2': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_3': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_4': 'aTAL',
            'HighMT_Doub_ATL|DTL_5': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_6': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_7': 'HighMT_Doub_TAL|ATL',
            'HighMT_Doub_ATL|DTL_8': 'HighMT_Doub_DCT|ATL',
            'HighMT_LowQ_PT_0': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_1': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_2': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_3': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_4': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_5': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT_6': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_0': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_1': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_2': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_3': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_4': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_5': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_6': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_7': 'HighMT_LowQ_PT',
            'HighMT_LowQ_PT|B cells_8': 'HighMT_LowQ_PT',
            'HighMT_Neural cells_0': 'Neuron',
            'HighMT_Neural cells_1': 'Neuron',
            'HighMT_Neural cells_2': 'LowQ_Neuron',
            'HighMT_Neural cells_3': 'LowQ_Neuron',
            'HighMT_Neural cells_4': 'LowQ_Neuron',
            'HighMT_Neural cells_5': 'LowQ_Neuron',
            'HighMT_PT/MD_0': 'HighMT_LowQ_PT',
            'HighMT_PT/MD_1': 'HighMT_LowQ_PT',
            'HighMT_PT/MD_2': 'HighMT_LowQ_PT',
            'HighMT_PT/MD_3': 'HighMT_LowQ_PT',
            'HighMT_PT/MD_4': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_0': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_1': 'PTS1/S2',
            'HighMT_PTS1/2_2': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_3': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_4': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_5': 'HighMT_LowQ_PT',
            'HighMT_PTS1/2_6': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_0': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_1': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_2': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_3': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_4': 'HighMT_LowQ_PT',
            'HighMT_PTS1|EC_5': 'HighMT_LowQ_PT',
            'HighMT_PT_IL32_0': 'HighMT_LowQ_PT',
            'HighMT_PT_IL32_1': 'HighMT_LowQ_PT',
            'HighMT_PT_IL32_2': 'HighMT_LowQ_PT',
            'HighMT_PT_IL32_3': 'HighMT_LowQ_PT',
            'HighMT_PT_IL32_4': 'HighMT_LowQ_PT',
            'HighMT_PT|Macro_0': 'HighMT_LowQ_PT',
            'HighMT_PT|Macro_1': 'HighMT_LowQ_PT',
            'HighMT_PT|Macro_2': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_0': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_1': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_2': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_3': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_4': 'HighMT_LowQ_PT',
            'HighMT_TAL|EC_5': 'HighMT_LowQ_PT',
            'TAL_0': 'TAL',
            'TAL_1': 'TAL',
            'TAL_2': 'TAL',
            'TAL_3': 'CNT-PC',
            'TAL_4': 'TAL',
            'TAL_5': 'ATL|CD-PC',
            'TAL_6': 'TAL',
            'TAL_7': 'TAL',
            'CNT-PC_0': 'DCT1',
            'CNT-PC_1': 'CNT-PC',
            'CNT-PC_2': 'CNT-PC',
            'CNT-PC_3': 'CNT-PC',
            'CNT-PC_4': 'DCT2',
            'CNT-PC_5': 'Doub_Immune|CNT-PC',
            'Damaged_urothelial_0': 'Doub_EC|TAL',
            'Damaged_urothelial_1': 'CD-PC',
            'Damaged_urothelial_2': 'aPTS1/S2',
            'Damaged_urothelial_3': 'Doub_ATL|CD',
            'Damaged_urothelial_4': 'Doub_EC|TAL',
            'Damaged_urothelial_5': 'Doub_EC|TAL',
            'Damaged_urothelial_6': 'CNT-PC',
            'Doub_CD-ICA|CD-ICB_0': 'CD-ICB',
            'Doub_CD-ICA|CD-ICB_1': 'CD-ICB',
            'Doub_CD-ICA|CD-ICB_2': 'dCD-ICB',
            'Doub_CD-ICA|CD-ICB_3': 'Doub_ATL|CD-ICB',
            'Doub_CD-ICA|CD-ICB_4': 'CD-ICB',
            'Doub_CD-ICA|CD-ICB_5': 'CD-ICB',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_0': 'CNT-PC',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_1': 'CD-ICA',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_2': 'CD-ICA',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_3': 'CNT-ICA',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_4': 'Doub_CNT-ICA|Macro',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_5': 'CD-PC',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_6': 'CNT-PC',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_7': 'CNT-ICA',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_8': 'Doub_EC|ICA',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_9': 'Doub_TAL|CD',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_10': 'Doub_EC|CNT',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_11': 'CD-PC',
            'Doub_CNT-IC-A|CD-ICA|CNT-PC_12': 'ATL',
            'CD-PC_0': 'CD-PC',
            'CD-PC_1': 'CNT-PC',
            'CD-PC_2': 'dCD-PC',
            'CD-PC_3': 'aCD-PC',
            'CD-PC_4': 'Doub_aPT|CD-PC',
            'CD-PC_5': 'CNT-PC',
            'CD-PC_6': 'CD-PC',
            'CD-PC_7': 'aCD-PC',
            'CD-PC_8': 'dCD-PC',
            'CD-PC_9': 'aCD-PC',
            'CD-PC_10': 'TAL',
            'IC-A_0': 'CD-ICA_Zhang',
            'IC-A_1': 'CD-ICA_Zhang',
            'IC-A_2': 'CD-ICA_Zhang',
            'IC-A_3': 'CD-ICA_Zhang',
            'IC-A_4': 'CD-ICA_Zhang',
            'IC-A_5': 'CD-ICA_Zhang',
            'IC-A_6': 'CD-ICA_Zhang',
            'IC-A_7': 'CD-ICA_Zhang',
            'Doub_PTS1/2|Fibro_0': 'Doub_aTAL|aPT',
            'Doub_PTS1/2|Fibro_1': 'Doub_ATL|PTS2',
            'Doub_PTS1/2|Fibro_2': 'Doub_PT|Macro',
            'Doub_PTS1/2|Fibro_3': 'Doub_aTAL|aPT',
            'Doub_PTS1/2|Fibro_4': 'aPT',
            'Doub_PTS1/2|Fibro_5': 'dPT',
            'Doub_PTS1/2|Fibro_6': 'Doub_VSMC|PT',
            'Doub_PTS1/2|Fibro_7': 'Doub_PT|Lympho',
            'Doub_PTS1/2|Fibro_8': 'Doub_VSMC|PT',
            'Doub_PTS1/2|Fibro_9': 'Doub_aTAL|aPT',
            'Doub_PTS1/2|Fibro_10': 'Doub_aTAL|aPT',
            'Doub_PTS1/2|Fibro_11': 'Doub_aTAL|aPT',
            'Doub_PTS1/2|Fibro_12': 'Doub_aTAL|Macro',
            'Doub_PEC|PT_0': 'aPT_Zhang',
            'Doub_PEC|PT_1': 'aPT_Zhang',
            'Doub_PEC|PT_2': 'aPT_Zhang',
            'Doub_PEC|PT_3': 'aPT_Zhang',
            'Doub_PEC|PT_4': 'aPT_Zhang',
            'Doub_PEC|PT_5': 'aPT_Zhang',
            'Doub_PEC|PT_6': 'aPT_Zhang',
            'Doub_PEC|PT_7': 'aPT_Zhang',
            'Doub_PEC|PT_8': 'aPT_Zhang',
            'PTS1_0': 'PTS1/S2',
            'PTS1_1': 'PTS2',
            'PTS1_2': 'PTS1',
            'PTS1_3': 'PTS1/S2_medullary',
            'PTS1_4': 'PTS1/S2',
            'PTS1_5': 'PTS1/S2',
            'PTS1_medulla_0': 'PTS1/S2_medullary',
            'PTS1_medulla_1': 'PTS1/S2_medullary',
            'PTS1_medulla_2': 'PTS3',
            'PTS1_medulla_3': 'PTS3',
            'PTS1_medulla_4': 'PTS3',
            'PTS1_medulla_5': 'PTS1/S2_medullary',
            'PTS3_0': 'PTS2/S3',
            'PTS3_1': 'aPTS1/S2',
            'PTS3_2': 'PTS2/S3',
            'PTS3_3': 'aPTS3',  # Immune cluster inside
            'PTS3_4': 'Doub_PTS1/S2|ATL',
            'PTS3_5': 'Doub_PTS1/S2|ATL',
            'PTS3_6': 'PTS2/S3',
            'PTS3_7': 'aPTS1/S2',
            'PTS3_8': 'Doub_PTS1/S2|Immune',
            'PTS3_9': 'Doub_PTS1/S2|Immune',
            'POD_0': 'POD',
            'POD_1': 'POD',
            'POD_2': 'POD',
            'POD_3': 'dPOD',
            'POD_4': 'dPOD',
            'POD_5': 'Phagocytic_POD_AIF1+',
            'POD_6': 'POD',
            'POD_7': 'Doub_EC|MC',
            'POD_8': 'Platelets',
            'PapE_0': 'PapE1',
            'PapE_1': 'PapE1',
            'PapE_2': 'PapE2',
            'PapE_3': 'PapE1',
            'PapE_4': 'PapE2',
            'PapE_5': 'PapE1',
            'PapE_6': 'PapE1',
            'PapE_7': 'PapE1',
            'PapE_8': 'dPapE',
            'PapE_9': 'LowQ_PapE',
            'PapE_10': 'LowQ_PapE',
            'PEC|Pancreatic_0': 'PEC',
            'PEC|Pancreatic_1': 'PEC',
            'PEC|Pancreatic_2': 'PEC',
            'PEC|Pancreatic_3': 'PEC',
            'PEC|Pancreatic_4': 'PEC',
            'PEC|Pancreatic_5': 'Doub_PEC|Immune',
            'PEC_0': 'MC',
            'PEC_1': 'Doub_EC|MC',
            'PEC_2': 'Doub_EC|MC',
            'PEC_3': 'Doub_EC|MC',
            'PEC_4': 'Doub_EC|MC',
            'LowQ_PT|EC_0': 'RBC',
            'LowQ_PT|EC_1': 'RBC',
            'LowQ_PT|EC_2': 'RBC',
            'LowQ_PT|EC_3': 'RBC',
            'LowQ_PT|EC_4': 'RBC',
            'LowQ_PT|EC_5': 'RBC',
            'LowQ_PT|EC_6': 'RBC',
            'LowQ_PT|EC_7': 'RBC',
            'LowQ_PT|EC_8': 'RBC',
            'LowQ_PT|EC_9': 'RBC',
            'LowQ_PT|EC_10': 'RBC',
            'LowQ_PT_0': 'LowQ_PT',
            'LowQ_PT_1': 'LowQ_PT',
            'LowQ_PT_2': 'LowQ_PT',
            'LowQ_0': 'PAC',
            'LowQ_1': 'PAC',
}

adata.obs['fine_annotation'] = adata.obs['leiden_R'].map(l_R_anno)

In [ ]:
sc.pl.embedding(adata, basis='X_scVI_umap', color=['fine_annotation'], ncols=1, palette=list(matplotlib.colors.CSS4_COLORS.values()))

In [ ]:
adata

In [ ]:
columns_to_drop = [
    'Immune_labels', 'predicted_labels', 'over_clustering', 'majority_voting', 'conf_score',
    'adip_cap_ec', 'aerocyte_ec', 'aorta_coronary_ec', 'aorta_coronary_smc', 'art_ec_1',
    'art_ec_2', 'art_smc', 'blood_brain_barrier_ec', 'brain_art_ec', 'brain_art_smc',
    'brain_ven_ec', 'bridging_lec', 'cap_ec', 'cap_lec', 'cap_pc', 'ceiling_lec',
    'endocardial_ec', 'endometrium_cap_ec', 'floor_lec', 'glomeruli_ec', 'kidney_art_ec',
    'kidney_cap_ec', 'littoral_EC', 'myo_cap_ec', 'pericentral_cap_ec',
    'perifollicular_sinus_lec', 'periportal_cap_ec', 'pul_art_ec', 'pul_cap_ec', 'pul_pc',
    'pul_smc', 'pul_tip_cap_ec', 'pul_ven_ec', 'smc_pc_intermediate', 'spleen_art_ec',
    'uterine_pc', 'uterine_smc', 'valve_down_lec', 'valve_up_lec', 'ven_ec_1', 'ven_ec_2',
    'ven_smc', 'score_PT', 'score_PTS1', 'score_PTS2', 'score_PTS3', 'score_PODCxG',
    'score_POD', 'score_PC', 'renal_PC', 'score_MD', 'score_DCT', 'score_PapE', 'score_CNT',
    'score_CNT-PC', 'score_CNT-IC-A', 'score_CD', 'score_CD-PC', 'score_CD-ICA',
    'score_ICB', 'score_LH', 'score_ATL', 'score_DTL', 'score_TAL', 'score_EC',
    'score_EC_AEA', 'score_EC_DVR', 'score_EC_PTC', 'score_EC_AVR', 'score_EC_LYM',
    'score_EC_CAP', 'score_MC', 'MC_CxG', 'score_VSMC', 'score_fibro', 'score_Macro',
    'score_Mono', 'score_neuro', 'score_DC', 'score_Lympho', 'score_Tnaive',
    'score_CD4Efmem', 'score_CD8Efmem', 'score_Treg', 'score_Tgd', 'score_TMAIT',
    'score_NKT', 'score_Th17', 'score_NK', 'score_ILC', 'score_Bcell', 'score_Plasmacell',
    'score_PEC', 'score_aTAL', 'score_aPT', 'score_EC-glomerular'
]

# Drop only if the column exists to avoid errors
adata.obs.drop(columns=[col for col in columns_to_drop if col in adata.obs.columns], inplace=True)

In [ ]:
# Load CSVs using the index column
immune = pd.read_csv('Downloads/immune_labels.csv', index_col=0)
vasc = pd.read_csv('Downloads/vasc_labels.csv', index_col=0)

# Reindex to align with adata.obs
adata.obs['Immune_pred'] = immune.reindex(adata.obs.index).iloc[:, 0]
adata.obs['Vasc_pred'] = vasc.reindex(adata.obs.index).iloc[:, 0]

In [ ]:
adata.write_h5ad('HCA_kidney_sc_annotated.h5ad')